In [4]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [5]:
import pandas as pd
from xbbg import blp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from datetime import timedelta as td
import statsmodels.api as sm
import re
import matplotlib.image as image
from ipywidgets import interact, IntSlider, Checkbox, Dropdown, Output, HBox, VBox, interactive, interactive_output, ToggleButton,Text, Button, DatePicker, IntText, ToggleButtons, RadioButtons,SelectMultiple
from IPython.display import display, clear_output
import itertools
from scipy import stats
from scipy.optimize import minimize 
from scipy.special import ndtr

c_freq = {'BTP':200,'DBR':100,'FRTR':100,'GGB':100, 'GILT':200, 'IRISH':100,'NETHER':100,'PGB':100,'RAGB':100,'SPGB':100,'UST':200}
scheds = {i: pd.read_excel(f"Y:\\Analytics and Tools\\SUPPLY AND NET SUPPLY\\Schedules (DO NOT MOVE)\\{i.lower()}_schedule.xlsx").rename(columns = {'Unnamed: 0':'Date'}).set_index('Date').fillna(0) for i in c_freq.keys()}
final_sched = {}
final_sched['USA'] = scheds['UST']
final_sched['UK'] = scheds['GILT']
final_sched['EU Core'] = pd.read_excel("Y:\\Analytics and Tools\\SUPPLY AND NET SUPPLY\\Schedules (DO NOT MOVE)\\EU Core_schedule.xlsx").rename(columns = {'Unnamed: 0':'Date'}).set_index('Date').fillna(0)
final_sched['EU Periph.'] = pd.read_excel("Y:\\Analytics and Tools\\SUPPLY AND NET SUPPLY\\Schedules (DO NOT MOVE)\\EU Periph._schedule.xlsx").rename(columns = {'Unnamed: 0':'Date'}).set_index('Date').fillna(0)

In [6]:
style = {'description_width': 'initial'}

tdy = dt.today()
year,month= tdy.year,tdy.month

_from = DatePicker(value = dt(year,month,1,0,0,0) ,description = 'From')
_to = DatePicker(value = dt(year,(month+1)%12,1,0,0,0) - td(1),description = 'To')
frequency = Dropdown(options =['Weekly','Monthly','Quarterly','Yearly'],description = 'Look at')
ecy = Dropdown(options =['USA','UK','EU Core', 'EU Periph.'],description = 'Economic Region',style = style)

output1 = Output()
produce_table = Button (description ='Produce Table', style=style) 

ui = VBox([HBox([ecy,frequency]),HBox([_from,_to])])



display(ui)
display(produce_table,output1)

def on_button_clicked1(b):
        with output1:
            clear_output()
            df = final_sched[ecy.value]
            df.index = pd.to_datetime(df.index)
            check_start = True
            check_end = True
            i = 0
            j = 0

            while check_start:
                if df.index[i]>=dt(_from.value.year,_from.value.month,_from.value.day):
                    start = df.index[i]
                    check_start =False
                i+=1

            while check_end:
                if df.index[j]>=dt(_to.value.year,_to.value.month,_to.value.day):
                    end = df.index[j]
                    check_end =False
                j+=1
                
            summ = df.loc[start:end].resample(frequency.value[0]).sum()
            if frequency.value == 'Yearly':
                summ.index = [i.strftime('%Y') for i in summ.index]
            elif frequency.value == 'Monthly':
                summ.index = [i.strftime('%b %y') for i in summ.index]
            elif frequency.value == 'Quarterly':
                 summ.index = [f"Q{i.month//3} {i.strftime('%y')}" for i in summ.index]
            else:
                summ.index = [i.strftime('%d %b %y') for i in summ.index]     
            if ecy.value == 'USA':
                display(summ.T.style.format('$ {:.1f} MM'))
            elif ecy.value == 'UK':
                display(summ.T.style.format('£ {:.1f} MM'))
            else:
                display(summ.T.style.format('€ {:.1f} MM'))
produce_table.on_click(on_button_clicked1)

Button(description='Produce Table', style=ButtonStyle())

Output()